In [2]:
!datamodel-codegen  --input carpurchase.json --input-file-type jsonschema --output model.py

In [3]:
from pathlib import Path
from tempfile import TemporaryDirectory
from datamodel_code_generator import InputFileType, generate
from datamodel_code_generator import DataModelType

json_schema: str = """{
    "type": "object",
    "properties": {
        "number": {"type": "number"},
        "street_name": {"type": "string"},
        "street_type": {"type": "string",
                        "enum": ["Street", "Avenue", "Boulevard"]
                        }
    }
}"""

with TemporaryDirectory() as temporary_directory_name:
    temporary_directory = Path(temporary_directory_name)
    output = Path(temporary_directory / 'model.py')
    generate(
        json_schema,
        input_file_type=InputFileType.JsonSchema,
        input_filename="example.json",
        output=output,
        # set up the output model types
        output_model_type=DataModelType.PydanticV2BaseModel,
    )
    model: str = output.read_text()
print(model)

# generated by datamodel-codegen:
#   filename:  example.json
#   timestamp: 2024-10-24T11:40:48+00:00

from __future__ import annotations

from enum import Enum
from typing import Optional

from pydantic import BaseModel


class StreetType(Enum):
    Street = 'Street'
    Avenue = 'Avenue'
    Boulevard = 'Boulevard'


class Model(BaseModel):
    number: Optional[float] = None
    street_name: Optional[str] = None
    street_type: Optional[StreetType] = None



In [8]:
from typing import Any, Dict, Type
from pydantic import create_model, BaseModel

def create_dynamic_model(
    model_name: str,
    fields: Dict[str, tuple[Type, Any]],
    base_class: Type[BaseModel] = BaseModel
) -> Type[BaseModel]:
    """
    Create a dynamic Pydantic model with specified fields.
    
    Args:
        model_name: Name of the model class
        fields: Dictionary of field names and their types/defaults
               Format: {"field_name": (type, default_value)}
        base_class: Base class to inherit from (defaults to BaseModel)
    
    Returns:
        A new Pydantic model class
    
    Example:
        fields = {
            "name": (str, ...),  # ... means required field
            "age": (int, 0),     # 0 is the default value
            "email": (str, None) # None is the default value
        }
        UserModel = create_dynamic_model("UserModel", fields)
    """
    return create_model(
        model_name,
        __base__=base_class,
        **fields
    )

# Example usage
def example_usage():
    # Define fields for a User model
    user_fields = {
        "name": (str, ...),
        "age": (int, 0),
        "email": (str, None),
        "is_active": (bool, True)
    }
    
    # Create the dynamic model
    UserModel = create_dynamic_model("UserModel", user_fields)
    
    # Create an instance
    user = UserModel(name="John Doe", email="john@example.com")
    return user

In [9]:
user = example_usage()

In [10]:
type(user)

__main__.UserModel

In [12]:
from pydantic import BaseModel, Field
from typing import Optional


class CarPurchase(BaseModel):
    full_name: str = Field(
        ...,
        description="The user's full name, which is required for personalizing the information. This must be validated by spelling it back to the user and asking for confirmation before using it in the function call.",
    )
    brand: str = Field(
        ...,
        description='The brand of the car the user is interested in. This is a required parameter.',
    )
    max_budget: Optional[float] = Field(
        ...,
        description="The maximum budget the user has for purchasing the car. This information is optional and must be asked to the user once, but proceed if the user doesn't want to provide it. Set it as 100000 if no info from the conversation can be inferred.",
    )
    current_time_of_year: Optional[str] = Field(
        ...,
        description="The time of the year when the user is inquiring about the car purchase, such as 'Spring', 'Summer', 'Fall', or 'Winter'. This should be inferred from the conversation context and not directly asked to the user. If no info from the conversation can be inferred, set it as 'Summer'.",
    )

In [14]:
car = CarPurchase(full_name='Aleix', brand="Bmw", max_budget=12344, current_time_of_year="summer")

In [15]:
type(car)

__main__.CarPurchase